# Whole-body manipulation
The objective of this exercise is to reach multiple targets while keeping balance in the Talos humanoid robot.
<img src="https://pal-robotics.com/wp-content/uploads/2024/04/TALOS_datasheets-rayos-768x1349.webp" alt="drawing" width="250"/>


This exercise focuses on a multi-contact optimal control problem of the form:

\begin{equation}\nonumber
	\begin{aligned}
		\min_{\mathbf{x}_s,\mathbf{u}_s}
		&\hspace{-2.em}
		& & \hspace{-0.75em}l_N(\mathbf{x}_{N})+\sum_{k=0}^{N-1} \int_{t_k}^{t_k+\Delta t_k}\hspace{-2.em} l_k(\mathbf{x}_k,\mathbf{u}_k)dt \hspace{-8.em}&\\
		& \hspace{-1em}\textrm{s.t.}
		& & \mathbf{q}_{k+1} = \mathbf{q}_k \oplus \int_{t_k}^{t_k+\Delta t_k}\hspace{-2.em}\mathbf{v}_{k+1}\,dt, &\textrm{(integrator)}\\
		& & & \mathbf{v}_{k+1} = \mathbf{v}_k + \int_{t_k}^{t_k+\Delta t_k}\hspace{-2.em}\mathbf{\dot{v}}_k\,dt, &\\
		& & & \hspace{-1em}\left[\begin{matrix}\mathbf{\dot{v}}_k \\ -\boldsymbol{\lambda}_k\end{matrix}\right] =
		\left[\begin{matrix}\mathbf{M} & \mathbf{J}^{\top}_c \\ {\mathbf{J}_{c}} & \mathbf{0} \end{matrix}\right]^{-1}
		\left[\begin{matrix}\boldsymbol{\tau}_b \\ -\mathbf{a}_0 \\\end{matrix}\right], &\textrm{(contact dynamics)}\\
		& & & \mathbf{R}\boldsymbol{\lambda}_{\mathcal{C}(k)} \leq \mathbf{\mathbf{r}}, &\textrm{(friction-cone)}\\
			& & & \mathbf{\bar{x}} \leq \mathbf{x}_k \leq \mathbf{\underline{x}}, &\textrm{(state bounds)}
		\end{aligned}
\end{equation}

where $l_i(\mathbf{x}_i, \mathbf{u}_i) = w_{hand}\|\log{(\mathbf{p}_{\mathcal{G}(k)}(\mathbf{q}_k)^{-1} \mathbf{^oM}_{\mathbf{f}_{\mathcal{G}(k)}})}\| + w_{xreg}\|\mathbf{x} - \mathbf{x}_0\|_{Q} + w_{ureg}\|\mathbf{u}\|_{R}$. Note that (1) the first term is the hand placement cost and (2) the terminal cost does not include the control regularization term.

Below there is a basic example that defines the above problem for reaching one target. Later, you will have to build the problem on top of it.

Without no more preamble, let's first declare the robot model and the foot and hand names!

In [ ]:
import crocoddyl
import example_robot_data
import numpy as np
import pinocchio as pin

# Load robot
robot = example_robot_data.load("talos")
rmodel = robot.model
q0 = rmodel.referenceConfigurations["half_sitting"]
x0 = np.concatenate([q0, np.zeros(rmodel.nv)])

# Declaring the foot and hand names
rf_name = "right_sole_link"
lf_name = "left_sole_link"
lh_name = "gripper_left_joint"

# Getting the frame ids
rf_id = rmodel.getFrameId(rf_name)
lf_id = rmodel.getFrameId(lf_name)
lh_id = rmodel.getFrameId(lh_name)

# Define the robot's state and actuation
state = crocoddyl.StateMultibody(rmodel)
actuation = crocoddyl.ActuationModelFloatingBase(state)

With the following function, we could build a differential action model giving a desired hand target.
The function builds a double-support contact phase and defines a hand-placement task. The cost function also includes:
 - state and control regularization terms
 - state limits penalization
 - friction cone penalization


In [ ]:
def createActionModel(target = None):
    # Creating a double-support contact (feet support)
    contacts = crocoddyl.ContactModelMultiple(state, actuation.nu)
    lf_contact = crocoddyl.ContactModel6D(
        state,
        lf_id,
        pin.SE3.Identity(),
        pin.LOCAL_WORLD_ALIGNED,
        actuation.nu,
        np.array([0, 40.]),
    )
    rf_contact = crocoddyl.ContactModel6D(
        state,
        rf_id,
        pin.SE3.Identity(),
        pin.LOCAL_WORLD_ALIGNED,
        actuation.nu,
        np.array([0, 40.]),
    )
    contacts.addContact("lf_contact", lf_contact)
    contacts.addContact("rf_contact", rf_contact)

    # Define the cost sum (cost manager)
    costs = crocoddyl.CostModelSum(state, actuation.nu)

    # Adding the hand-placement cost
    if target is not None:
        w_hand = np.array([1] * 3 + [0.0001] * 3)
        lh_Mref = pin.SE3(np.eye(3), target)
        activation_hand = crocoddyl.ActivationModelWeightedQuad(w_hand**2)
        lh_cost = crocoddyl.CostModelResidual(
            state,
            activation_hand,
            crocoddyl.ResidualModelFramePlacement(state, lh_id, lh_Mref, actuation.nu),
        )
        costs.addCost("lh_goal", lh_cost, 1e2)

    # Adding state and control regularization terms
    w_x = np.array([0] * 3 + [10.0] * 3 + [0.01] * (state.nv - 6) + [10] * state.nv)
    activation_xreg = crocoddyl.ActivationModelWeightedQuad(w_x**2)
    x_reg_cost = crocoddyl.CostModelResidual(
        state, activation_xreg, crocoddyl.ResidualModelState(state, x0, actuation.nu)
    )
    u_reg_cost = crocoddyl.CostModelResidual(
        state, crocoddyl.ResidualModelControl(state, actuation.nu)
    )
    costs.addCost("xReg", x_reg_cost, 1e-3)
    costs.addCost("uReg", u_reg_cost, 1e-4)

    # Adding the state limits penalization
    x_lb = np.concatenate([state.lb[1 : state.nv + 1], state.lb[-state.nv :]])
    x_ub = np.concatenate([state.ub[1 : state.nv + 1], state.ub[-state.nv :]])
    activation_xbounds = crocoddyl.ActivationModelQuadraticBarrier(
        crocoddyl.ActivationBounds(x_lb, x_ub)
    )
    x_bounds = crocoddyl.CostModelResidual(
        state,
        activation_xbounds,
        crocoddyl.ResidualModelState(state, actuation.nu),
    )
    costs.addCost("xBounds", x_bounds, 1.0)

    # Adding the friction cone penalization
    nsurf, mu = np.identity(3), 0.7
    cone = crocoddyl.FrictionCone(nsurf, mu, 4, False)
    activation_friction = crocoddyl.ActivationModelQuadraticBarrier(
        crocoddyl.ActivationBounds(cone.lb, cone.ub)
    )
    lf_friction = crocoddyl.CostModelResidual(
        state,
        activation_friction,
        crocoddyl.ResidualModelContactFrictionCone(state, lf_id, cone, actuation.nu),
    )
    rf_friction = crocoddyl.CostModelResidual(
        state,
        activation_friction,
        crocoddyl.ResidualModelContactFrictionCone(state, rf_id, cone, actuation.nu),
    )
    costs.addCost("lf_friction", lf_friction, 1e1)
    costs.addCost("rf_friction", rf_friction, 1e1)

    # Creating the action model
    dmodel = crocoddyl.DifferentialActionModelContactFwdDynamics(
        state, actuation, contacts, costs
    )
    return dmodel

Finally, the following function allows us to display the motions and desired targets:

In [ ]:
import meshcat.geometry as g

def createDisplay(targets):
    display = crocoddyl.MeshcatDisplay(robot, 4, 4, False)
    for i, target in enumerate(targets):
        display.robot.viewer["target_" + str(i)].set_object(g.Sphere(0.05))
        display.robot.viewer["target_" + str(i)].set_transform(
            np.array(
                [
                    [1.0, 0.0, 0.0, target[0]],
                    [0.0, 1.0, 0.0, target[1]],
                    [0.0, 0.0, 1.0, target[2]],
                    [0.0, 0.0, 0.0, 1.0],
                ]
            )
        )
    return display

Now, we create an optimal control problem to reach a single target

In [ ]:
DT, N = 5e-2, 20
target = np.array([0.4, 0, 1.2])

# Creating a running model for the target
running_models = [crocoddyl.IntegratedActionModelEuler(createActionModel(target), DT)] * N
terminal_model = crocoddyl.IntegratedActionModelEuler(createActionModel(target), 0.0)
print("Running models:", running_models[0])
print("Terminal model:", terminal_model)


# Defining the problem and the solver
problem = crocoddyl.ShootingProblem(x0, running_models, terminal_model)
fddp = crocoddyl.SolverFDDP(problem)

# Creating display
display = createDisplay([target])

# Adding callbacks to inspect the evolution of the solver (logs are printed in the terminal)
fddp.setCallbacks([crocoddyl.CallbackVerbose()])

Let's solve this problem!

In [ ]:
status = fddp.solve()
print("\nProblem solved:", status, flush=True)
print("Number of iterations:", fddp.iter)
print("Total cost:", fddp.cost)
print("Gradient norm:", fddp.stoppingCriteria())

# Embedded in this cell
display.robot.viewer.jupyter_cell()

You could display again the final solution

In [ ]:
display.rate = -1
display.freq = 1

display.displayFromSolver(fddp)

## Modifying the example

Let's build an optimal control problem to reach 4 targets as described below:

In [ ]:
targets = []
targets += [np.array([0.4, 0.1, 1.2])]
targets += [np.array([0.6, 0.1, 1.2])]
targets += [np.array([0.6, -0.1, 1.2])]
targets += [np.array([0.4, -0.1, 1.2])]

Now let's display the targets in Meshcat. Do not forget to embed again the display into the jupyter cell

In [ ]:
# Embedded in this cell
display.robot.viewer.jupyter_cell()

After checking that everything is alright, it's time to build the sequence!
Do not forget to create the problem as well :)

And we solve it as before

In [ ]:
# Create the FDDP solver
fddp = crocoddyl.SolverFDDP(problem)
fddp.setCallbacks([crocoddyl.CallbackVerbose(), crocoddyl.CallbackDisplay(display)])

# Solves the problem
print("Problem solved:", fddp.solve())
print("Number of iterations:", fddp.iter)
print("Total cost:", fddp.cost)
print("Gradient norm:", fddp.stoppingCriteria())

Do not miss the change to display the motion at the right display speed!

In [ ]:
display.rate = -1
display.freq = 1
display.displayFromSolver(fddp)

## Same targets with right hand

You've learned how to reach 4 targets with the left hand, congratulations!

To keep playing within this problem, you should create a new createActionModel to achieve the same task for the right hand.

In [ ]:
def createActionModel(target):
    # now god is with you xD
    # time to show you up!

And here you need to create the problem and solve.
Do not forget to display the results